<a href="https://colab.research.google.com/github/NuCapybara/BouncingJack/blob/main/Final_Project_ME314.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## HELPER FUNCTIONS
def hat(omg):
    """Converts a 3-vector to an so(3) representation

    :param omg: A 3-vector
    :return: The skew symmetric representation of omg

    Example Input:
        omg = np.array([1, 2, 3])
    Output:
        np.array([[ 0, -3,  2],
                  [ 3,  0, -1],
                  [-2,  1,  0]])
    """
    return sym.Matrix([[0,      -omg[2],  omg[1]],
                     [omg[2],       0, -omg[0]],
                     [-omg[1], omg[0],       0]])

def unhat(mat):
    """Converts an SE(3) representation to a 6-vector
    """
    return sym.Matrix([mat[0, 3], mat[1, 3], mat[2, 3], mat[2, 1], mat[0, 2], mat[1, 0]])

def SE3generator_np(x, y, theta):
    """
    This function return SE(3) given x, y, theta
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, x],
                     [np.sin(theta),  np.cos(theta), 0, y],
                     [            0,              0, 1, 0],
                     [            0,              0, 0, 1]])


def SE3generator_sym(x, y, theta):
  """
  This function return SE(3) given x, y, theta
  """
  return sym.Matrix([[sym.cos(theta), -sym.sin(theta), 0, x],
                    [sym.sin(theta),  sym.cos(theta), 0, y],
                    [            0,              0, 1, 0],
                    [            0,              0, 0, 1]])

def mat_inverse(g):
    R_inv = g[:3, :3].T
    p_inv = -R_inv * g[:3, 3]
    return sym.Matrix([[R_inv[0, 0], R_inv[0, 1], R_inv[0, 2], p_inv[0]],
                   [R_inv[1, 0], R_inv[1, 1], R_inv[1, 2], p_inv[1]],
                   [R_inv[2, 0], R_inv[2, 1], R_inv[2, 2], p_inv[2]],
                   [0, 0, 0, 1]])



def integrate(f, xt, dt):
    """
    This function takes in an initial condition x(t) and a timestep dt,
    as well as a dynamical system f(x) that outputs a vector of the
    same dimension as x(t). It outputs a vector x(t+dt) at the future
    time step.

    Parameters
    ============
    dyn: Python function
        derivate of the system at a given step x(t),
        it can considered as \dot{x}(t) = func(x(t))
    xt: NumPy array
        current step x(t)
    dt:
        step size for integration

    Return
    ============
    new_xt:
        value of x(t+dt) integrated from x(t)
    """
    k1 = dt * f(xt)
    k2 = dt * f(xt+k1/2.)
    k3 = dt * f(xt+k2/2.)
    k4 = dt * f(xt+k3)
    new_xt = xt + (1/6.) * (k1+2.0*k2+2.0*k3+k4)
    return new_xt

In [2]:
import sympy as sym
import numpy as np

In [3]:
#define variables
t = sym.symbols('t')

#physical property
frame_l = 4
j_l = 0.5
m_j_point = 1
Mf = 20
gravity = 9.8

#q defination
x_f = sym.Function('x_f')(t)
y_f = sym.Function('y_f')(t)
x_fdot = x_f.diff(t)
y_fdot = y_f.diff(t)
x_fddot = x_fdot.diff(t)
y_fddot = y_fdot.diff(t)
theta_f= sym.Function('θ_f')(t)
theta_fdot = theta_f.diff(t)
theta_fddot = theta_fdot.diff(t)

x_j = sym.Function('x_j')(t)
y_j = sym.Function('y_j')(t)
x_jdot = x_j.diff(t)
y_jdot = y_j.diff(t)
x_jddot = x_jdot.diff(t)
y_jddot = y_jdot.diff(t)
theta_j = sym.Function('θ_j')(t)
theta_jdot = theta_j.diff(t)
theta_jddot = theta_jdot.diff(t)

#define the frames of jack
g_wj = SE3generator_sym(x_j, y_j, theta_j)
g_jb = SE3generator_sym(j_l, 0, 0)
g_ja = SE3generator_sym(0, j_l, 0)
g_jc = SE3generator_sym(0, -j_l, 0)
g_jd = SE3generator_sym(-j_l, 0, 0)

g_wb = sym.simplify(g_wj * g_jb)
g_wa = sym.simplify(g_wj * g_ja)
g_wc = sym.simplify(g_wj * g_jc)
g_wd = sym.simplify(g_wj * g_jd)

#define the frame of the world
g_wf = SE3generator_sym(x_f, y_f, theta_f)
g_f6 = SE3generator_sym(frame_l/2, frame_l/2, 0)
g_f7 = SE3generator_sym(frame_l/2, -frame_l/2, 0)
g_f8 = SE3generator_sym(-frame_l/2, -frame_l/2, 0)
g_f9 = SE3generator_sym(-frame_l/2, frame_l/2, 0)

g_w6 = sym.simplify(g_wf * g_f6)
g_w7 = sym.simplify(g_wf * g_f7)
g_w8 = sym.simplify(g_wf * g_f8)
g_w9 = sym.simplify(g_wf * g_f9)

print("g_wf =")
display(g_wf)
print("g_wj =")
display(g_wj)

g_wf =


Matrix([
[cos(θ_f(t)), -sin(θ_f(t)), 0, x_f(t)],
[sin(θ_f(t)),  cos(θ_f(t)), 0, y_f(t)],
[          0,            0, 1,      0],
[          0,            0, 0,      1]])

g_wj =


Matrix([
[cos(θ_j(t)), -sin(θ_j(t)), 0, x_j(t)],
[sin(θ_j(t)),  cos(θ_j(t)), 0, y_j(t)],
[          0,            0, 1,      0],
[          0,            0, 0,      1]])

In [ ]:
### Inertia value of Jack and Frame
J_jack = 0
J_frame = Mf*(frame_l**2)/3

### Velocity & Inertia
  # Jack four point velocity
v_wa = sym.simplify(unhat(mat_inverse(g_wa)*g_wa.diff(t)))
v_wb = sym.simplify(unhat(mat_inverse(g_wb)*g_wb.diff(t)))
v_wc = sym.simplify(unhat(mat_inverse(g_wc)*g_wc.diff(t)))
v_wd = sym.simplify(unhat(mat_inverse(g_wd)*g_wd.diff(t)))


  # Frame four point velocity
v_wf = sym.simplify(unhat(mat_inverse(g_wf)*g_wf.diff(t)))
v_w6 = sym.simplify(unhat(mat_inverse(g_w6)*g_w6.diff(t)))
v_w7 = sym.simplify(unhat(mat_inverse(g_w7)*g_w7.diff(t)))
v_w8 = sym.simplify(unhat(mat_inverse(g_w8)*g_w8.diff(t)))
v_w9 = sym.simplify(unhat(mat_inverse(g_w9)*g_w9.diff(t)))

  #Inertia frame
Inertia_frame_mat = sym.diag(Mf, Mf, Mf, 0, 0, J_frame)
Inertia_jack_mat = sym.diag(m_j_point, m_j_point, m_j_point, 0, 0, J_jack)


#Kinetic Energy and Potential Energy
  #frame kinetic energy
KE_frame = sym.simplify(0.5*(v_wf).T*Inertia_frame_mat*v_wf)
  #jack kinetic energy
KE_vwa = sym.simplify(0.5*(v_wa).T*Inertia_jack_mat*v_wa)
KE_vwb = sym.simplify(0.5*(v_wb).T*Inertia_jack_mat*v_wb)
KE_vwc = sym.simplify(0.5*(v_wc).T*Inertia_jack_mat*v_wc)
KE_vwd = sym.simplify(0.5*(v_wd).T*Inertia_jack_mat*v_wd)

KE_jack = sym.simplify(KE_vwa + KE_vwb + KE_vwc + KE_vwd)

KE_total = sym.simplify(KE_jack+ KE_frame)
print("KE =")
display(KE_total)
hf = sym.simplify(sym.Matrix([g_wf[1,3]]))
h_wa = sym.simplify(sym.Matrix([g_wa[1,3]]))
h_wb = sym.simplify(sym.Matrix([g_wb[1,3]]))
h_wc = sym.simplify(sym.Matrix([g_wc[1,3]]))
h_wd = sym.simplify(sym.Matrix([g_wd[1,3]]))

V_f = Mf*gravity*hf[0]
V_wa =  m_j_point*gravity*h_wa[0]
V_wb =  m_j_point*gravity*h_wb[0]
V_wc =  m_j_point*gravity*h_wc[0]
V_wd =  m_j_point*gravity*h_wd[0]

V_total = sym.simplify(sym.Matrix([V_f + V_wa + V_wb + V_wc + V_wd]))
print("V =")
display(V_total)




In [ ]:
#Lagrangian Equation
L_eq = sym.simplify(KE_total - V_total)
print("The Lagrangian equation is")
lhs = L_eq[0]
rhs = 0
equ = sym.Eq(lhs, rhs)
display(equ)

#Euler Lagrangian, rhs is the upward force
Force = Mf * gravity + 39
jack_torque = 0
Force_Matrix = sym.Matrix([0, Force, 30, 0, 0, jack_torque])

q = sym.Matrix([x_f, y_f, theta_f, x_j, y_j, theta_j])
qdot = sym.simplify(q.diff(t))
qddot = sym.simplify(qdot.diff(t))

dLdqdot = sym.simplify(sym.Matrix([L_eq]).jacobian(qdot).T)
dLdqdot_dt = sym.simplify(dLdqdot.diff(t))
dLdq = sym.simplify(sym.Matrix([L_eq]).jacobian(q).T)
EoM_lhs = sym.simplify(dLdqdot_dt - dLdq)
EoM_rhs = Force_Matrix
EoM= sym.Eq(EoM_lhs, EoM_rhs)
print("Euler Lagrangian equation is:")
display(EoM)


In [ ]:
#Solution of Motion of Equation
q_sol = sym.Matrix([x_fddot, y_fddot, theta_fddot, x_jddot, y_jddot, theta_jddot])
Sol= sym.solve(EoM, q_sol, dict = True)
list = []
for sol in Sol:
  for v in q_sol:
    list.append(sym.N(sol[v]))
    display(sym.Eq(v, sym.simplify(sol[v])))

In [ ]:
#Get point locations


ra = sym.Matrix((g_wa @ sym.Matrix([0, 0, 0, 1]))[0:3])
rb = sym.Matrix((g_wb @ sym.Matrix([0, 0, 0, 1]))[0:3])
rc = sym.Matrix((g_wc @ sym.Matrix([0, 0, 0, 1]))[0:3])
rd = sym.Matrix((g_wd @ sym.Matrix([0, 0, 0, 1]))[0:3])

r6 = sym.Matrix((g_w6 @ sym.Matrix([0, 0, 0, 1]))[0:3])
r7 = sym.Matrix((g_w7 @ sym.Matrix([0, 0, 0, 1]))[0:3])
r8 = sym.Matrix((g_w8 @ sym.Matrix([0, 0, 0, 1]))[0:3])
r9 = sym.Matrix((g_w9 @ sym.Matrix([0, 0, 0, 1]))[0:3])

#Get connecting vectors between jack and frame

    #Point a
r6a = sym.simplify(ra - r6)
r7a = sym.simplify(ra - r7)
r8a = sym.simplify(ra - r8)
r9a = sym.simplify(ra - r9)

    #Point b
r6b = sym.simplify(rb - r6)
r7b = sym.simplify(rb - r7)
r8b = sym.simplify(rb - r8)
r9b = sym.simplify(rb - r9)

    #Point c
r6c = sym.simplify(rc - r6)
r7c = sym.simplify(rc - r7)
r8c = sym.simplify(rc - r8)
r9c = sym.simplify(rc - r9)

    #Point d
r6d = sym.simplify(rd - r6)
r7d = sym.simplify(rd - r7)
r8d = sym.simplify(rd - r8)
r9d = sym.simplify(rd - r9)

    #frame
r76 = sym.simplify(r6 - r7)
r87 = sym.simplify(r7 - r8)
r98 = sym.simplify(r8 - r9)
r96 = sym.simplify(r6 - r9)
r67 = sym.simplify(r7 - r6)
r78 = sym.simplify(r8 - r7)
r89 = sym.simplify(r9 - r8)
r69 = sym.simplify(r9 - r6)

In [ ]:
#Define 16 phi constraints

    #a point
phi_1 = r6a.cross(r67)[2]  #phi_a_67
phi_2 = r7a.cross(r78)[2]  #phi_a_78
phi_3 = r8a.cross(r89)[2]  #phi_a_89
phi_4 = r9a.cross(r96)[2]  #phi_a_96

    #b point
phi_5 = r6b.cross(r67)[2]    #phi_b_67
phi_6 = r7b.cross(r78)[2]    #phi_b_78
phi_7 = r8b.cross(r89)[2]    #phi_b_89
phi_8 = r9b.cross(r96)[2]    #phi_b_96

    #c point
phi_9 = r6c.cross(r67)[2]    #phi_c_67
phi_10 = r7c.cross(r78)[2]    #phi_c_78
phi_11 = r8c.cross(r89)[2]    #phi_c_89
phi_12 = r9c.cross(r96)[2]    #phi_c_96

    #d point
phi_13 = r6d.cross(r67)[2]    #phi_d_67
phi_14 = r7d.cross(r78)[2]    #phi_d_78
phi_15 = r8d.cross(r89)[2]    #phi_d_89
phi_16 = r9d.cross(r96)[2]    #phi_d_96

Phi_List = [phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7, phi_8, phi_9, phi_10, phi_11, phi_12, phi_13, phi_14, phi_15, phi_16]


In [ ]:
#REVISE!!!
# define dummy variables for q(tau-), qdot(tau-), qdot(tau+)
# note that we don’t need to define q(tau+), why? because q(tau+) = q(tau-)

#theta_f tau
theta_fMinus, theta_fdotMinus = sym.symbols(r'θ_f^-, \dot{θ_f}^-’')
theta_fdotPlus = sym.symbols(r'\dot{θ_f}^+')

#theta_j tau: theta_j wont change between tau+ tau-, so we will use tau- for theta_j
theta_jMinus, theta_jdotMinus = sym.symbols(r'θ_j^-, \dot{θ_j}^-’')
theta_jdotPlus = sym.symbols(r'\dot{θ_j}^+')

#x_j tau: x wont change between tau+ tau-, so we will use tau- for x
x_jMinus, x_jdotMinus = sym.symbols(r'x_j^-, \dot{x_j}^-’')
x_jdotPlus = sym.symbols(r'\dot{x_j}^+')

#x_f tau: x wont change between tau+ tau-, so we will use tau- for x
x_fMinus, x_fdotMinus = sym.symbols(r'x_f^-, \dot{x_f}^-’')
x_fdotPlus = sym.symbols(r'\dot{x_f}^+')

#y_j tau: x wont change between tau+ tau-, so we will use tau- for y
y_jMinus, y_jdotMinus = sym.symbols(r'y_j^-, \dot{y_j}^-’')
y_jdotPlus = sym.symbols(r'\dot{y_j}^+')

#y_f tau: x wont change between tau+ tau-, so we will use tau- for y
y_fMinus, y_fdotMinus = sym.symbols(r'y_f^-, \dot{y_f}^-’')
y_fdotPlus = sym.symbols(r'\dot{y_f}^+')


# define substitution dictionary to evalute expressions at tau- and tau+
# note that in "subs_plus", we replace q[0] and q[1] with xMinus and yMinus(why?)
subs_minus = {qdot[0]:x_fdotMinus, qdot[1]:y_fdotMinus,
            qdot[2]:theta_fdotMinus, qdot[3]:x_jdotMinus,
              qdot[4]:y_jdotMinus, qdot[5]:theta_jdotMinus}

subs_plus = {qdot[0]:x_fdotPlus, qdot[1]:y_fdotPlus,
              qdot[2]:theta_fdotPlus, qdot[3]:x_jdotPlus,
              qdot[4]:y_jdotPlus, qdot[5]:theta_jdotPlus}
# compute dLdqdot and evaluate it at tau- and tau+
dLdqdot = sym.Matrix([L_eq]).jacobian(qdot)
dLdqdot_Minus = sym.simplify(dLdqdot.subs(subs_minus))
dLdqdot_Plus = sym.simplify(dLdqdot.subs(subs_plus))



In [ ]:
#dPhidq_list build up
#dPhidqMinus_list build up

dPhidq_list = []
# dPhidq_Minus_list = []
for i in range(len(Phi_List)):
    phi_in_loop = Phi_List[i] #corresponding to phi_i+1, since i starting 0
    dPhidq_in_loop = sym.simplify(sym.Matrix([phi_in_loop]).jacobian(q))
    # dPhidqMinus_in_loop = sym.simplify(dPhidq_in_loop.subs(subs_minus))
    dPhidq_list.append(dPhidq_in_loop)
    # dPhidq_Minus_list.append(dPhidqMinus_in_loop)


In [ ]:
# Calculate Hamiltonian
H = sym.simplify((dLdqdot*qdot)[0]-L_eq[0])
H_Minus =sym.simplify(H.subs(subs_minus))
H_Plus = sym.simplify(H.subs(subs_plus))

print("\n\033[1mdL/dqdot = \033[0m")
display(sym.simplify(dLdqdot).T)
print("\n\033[1m H = \033[0m")
display(sym.simplify(H))

In [ ]:
# define equations for impact update
lamb = sym.symbols('λ')
impact_eqns_lhs = sym.Matrix([dLdqdot_Plus[0]-dLdqdot_Minus[0], dLdqdot_Plus[1]-dLdqdot_Minus[1], dLdqdot_Plus[2]-dLdqdot_Minus[2],
                              dLdqdot_Plus[3]-dLdqdot_Minus[3], dLdqdot_Plus[4]-dLdqdot_Minus[4], dLdqdot_Plus[5]-dLdqdot_Minus[5], H_Plus-H_Minus ])
impact_eqns_List = []
# display(dPhidq_Minus_list[2][0])
for i in range(len(Phi_List)):
    impact_eqns_rhs = sym.Matrix([lamb*dPhidq_list[i][0], lamb*dPhidq_list[i][1], lamb*dPhidq_list[i][2],
                                  lamb*dPhidq_list[i][3], lamb*dPhidq_list[i][4], lamb*dPhidq_list[i][5], 0])
    impact_eqns_loop = sym.Eq(impact_eqns_lhs, impact_eqns_rhs)
    impact_eqns_List.append(impact_eqns_loop)
# display(impact_eqns_List)

In [ ]:
###Helper Function Impact_update

def impact_update(state, index):

  #Define qplusdot, which is all the dot variable but at Tau+
  qplusdot = sym.Matrix([x_fdotPlus, y_fdotPlus, theta_fdotPlus, x_jdotPlus, y_jdotPlus, theta_jdotPlus, lamb])
  #Define all tau- variables,
  x_fminus= state[0]
  y_fminus = state[1]
  theta_fminus = state[2]
  x_jminus= state[3]
  y_jminus = state[4]
  theta_jminus = state[5]

  x_fdotminus = state[6]
  y_fdotminus = state[7]
  theta_fdotminus = state[8]
  x_jdotminus= state[9]
  y_jdotminus = state[10]
  theta_jdotminus = state[11]

  impact_equ_sub = impact_eqns_List[index].subs([(x_f, x_fminus),(y_f, y_fminus), (theta_f, theta_fminus),
                                      (x_j, x_jminus),(y_j, y_jminus), (theta_j, theta_jminus),
                                      (x_fdotMinus, x_fdotminus), (y_fdotMinus, y_fdotminus), (theta_fdotMinus, theta_fdotminus),
                                      (x_jdotMinus, x_jdotminus), (y_jdotMinus, y_jdotminus), (theta_jdotMinus, theta_jdotminus)])

  Sol_after_impact= sym.solve(impact_equ_sub, qplusdot, dict = True)


  list7 = []
  for sol in Sol_after_impact:
    if abs(sym.N(sol[lamb])) < 1e-2:
      continue

    for v in qplusdot:
      list7.append(sym.N(sol[v]))

  new_state = np.array([x_fminus, y_fminus, theta_fminus, x_jminus, y_jminus, theta_jminus,
                        list7[0],  list7[1],  list7[2], list7[3], list7[4], list7[5]])

  return new_state




In [ ]:
#Solve Impact Function
dLdqdot_dt = sym.simplify(dLdqdot.jacobian([t]))
dLdq = sym.simplify(L_eq[0].diff(q))

Euler_Lagrange_q = sym.simplify(dLdqdot_dt - dLdq)

rhs = sym.Matrix([0, Force, 30, 0, 0, 0])
Before_Impact_Equ = sym.Eq(Euler_Lagrange_q, rhs)
Soln8 = sym.solve(Before_Impact_Equ, qddot, dict = True)


list8 = []
for sol in Soln8:

  for v in qddot:
    list8.append(sym.lambdify([q[0], q[1], q[2], q[3], q[4], q[5], qdot[0],
                                     qdot[1], qdot[2], qdot[3], qdot[4], qdot[5]], sol[v]))
    display(sym.Eq(v, sym.simplify(sol[v])))

In [ ]:
#define the solution_functions:
x_fddotfunction = list8[0]
y_fddotfunction = list8[1]
theta_fddotfunction = list8[2]
x_jddotfunction = list8[3]
y_jddotfunction = list8[4]
theta_jddotfunction = list8[5]

#Lamdify Phi and lambdify them
phi_func_lamb_list = []

for i in range(len(Phi_List)):
  phi_func_lamb = sym.lambdify([q[0], q[1], q[2], q[3], q[4], q[5], qdot[0],
                                     qdot[1], qdot[2], qdot[3], qdot[4], qdot[5]], Phi_List[i])
  phi_func_lamb_list.append(phi_func_lamb)
# display(phi_func_lamb_list)



# function for impact condition
# since we are numerically simulating the system, when impact happends, phi(
# will not be exactly zero, thus we need to either catch the change of sign
# phi(q) or set a threshold for that

def impact_condition_jack(s,  phi_func_lamdified, threshold = 0.5):
  phi_val = phi_func_lamdified(*s)
  if phi_val > -threshold and phi_val < threshold:
    return True
  return False

def simulate_with_impact_jack(f, x0, tspan, dt, integrate):
  """
  simulate with impact
  """
  N = int((max(tspan)-min(tspan))/dt)
  x = np.copy(x0)
  tvec = np.linspace(min(tspan),max(tspan),N)
  xtraj = np.zeros((len(x0),N))
  for i in range(N):

    for j in range(16):
      if impact_condition_jack(x, phi_func_lamb_list[j]) is True:
        x = impact_update(x, j) #changed

    xtraj[:,i]=integrate(f,x,dt)
    x = np.copy(xtraj[:,i])
  return xtraj

### s is 12 variable vector, representing xf yf tf xj yj tj xfdot ....dot
def x_fddot1(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    a = x_fddotfunction(*s)
    return a

def y_fddot1(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    b = y_fddotfunction(*s)
    return b

def theta_fddot1(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    c = theta_fddotfunction(*s)
    return c

def x_jddot2(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    e = x_jddotfunction(*s)
    return e

def y_jddot2(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    f = y_jddotfunction(*s)
    return f

def theta_jddot2(s):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    g = theta_jddotfunction(*s)
    return g


def dyn(s):
    dyn_func = np.array([s[6], s[7], s[8], s[9], s[10], s[11],
                     x_fddot1(s), y_fddot1(s), theta_fddot1(s),
                     x_jddot2(s), y_jddot2(s), theta_jddot2(s)])
    return dyn_func


s0 = np.array([0, 0, 0, 0, 0, -np.pi/2, 0, 0, 0, 0, 0, 0])
traj_simulate = simulate_with_impact_jack(dyn, s0, [0,10], 0.01, integrate)
traj_list = np.array([traj_simulate[0], traj_simulate[1], traj_simulate[2],
                     traj_simulate[3], traj_simulate[4], traj_simulate[5]])

In [ ]:
# phi_func_lamb_list(13)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.linspace(0, 10, 1000), traj_simulate[0], label ='xf')
plt.plot(np.linspace(0, 10, 1000), traj_simulate[1], label ='yf')
plt.plot(np.linspace(0, 10, 1000), traj_simulate[2], label ='tf')
plt.plot(np.linspace(0, 10, 1000), traj_simulate[3], label ='xj')
plt.plot(np.linspace(0, 10, 1000), traj_simulate[4], label ='yj')
plt.plot(np.linspace(0, 10, 1000), traj_simulate[5], label ='tj')
plt.xlabel('Time')
plt.ylabel('Theta')
plt.title("Simulated trajectory of theta vs time")
plt.legend()


In [ ]:
def get_g_matrix(theta, px, py):
    return np.array([
        [   np.cos(theta), -np.sin(theta),  px],
        [   np.sin(theta),  np.cos(theta),  py],
        [   0,              0,              1 ]
    ])

def animate_system(theta_array,L1=1,L2=1,W=0.2,T=10):
    """
    Function to generate web-based animation of double-pendulum system

    Parameters:
    ================================================
    theta_array:
        trajectory of theta1 and theta2, should be a NumPy array with
        shape of (2,N)
    L1:
        length of the first pendulum
    L2:
        length of the second pendulum
    T:
        length/seconds of animation duration

    Returns: None
    """

    ################################
    # Imports required for animation.
    from plotly.offline import init_notebook_mode, iplot
    from IPython.display import display, HTML
    import plotly.graph_objects as go

    #######################
    # Browser configuration.
    def configure_plotly_browser_state():
        import IPython
        display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
              });
            </script>
            '''))
    configure_plotly_browser_state()
    init_notebook_mode(connected=False)
    N = len(theta_array[0])

    xxb = theta_array[0]
    yyb = theta_array[1]
    tb  = theta_array[2]
    xxj = theta_array[3]
    yyj = theta_array[4]
    tj  = theta_array[5]

    xb1 = np.zeros(N)
    yb1 = np.zeros(N)
    xb2 = np.zeros(N)
    yb2 = np.zeros(N)
    xb3 = np.zeros(N)
    yb3 = np.zeros(N)
    xb4 = np.zeros(N)
    yb4 = np.zeros(N)

    xj1 = np.zeros(N)
    yj1 = np.zeros(N)
    xj2 = np.zeros(N)
    yj2 = np.zeros(N)
    xj3 = np.zeros(N)
    yj3 = np.zeros(N)
    xj4 = np.zeros(N)
    yj4 = np.zeros(N)

    ###############################################
    # Define frames
    for i in range(N):
        # Box

        twa = get_g_matrix(0, xxb[i], yyb[i])
        tab = get_g_matrix(tb[i], 0, 0)
        tbc = get_g_matrix(0,  L1/2,  L1/2)
        tbd = get_g_matrix(0, -L1/2,  L1/2)
        tbe = get_g_matrix(0, -L1/2, -L1/2)
        tbf = get_g_matrix(0,  L1/2, -L1/2)

        twb = twa @ tab
        twc = twb @ tbc
        twd = twb @ tbd
        twe = twb @ tbe
        twf = twb @ tbf

        xb1[i] = np.array([1, 0, 0]) @ twc @ np.array([0, 0, 1])
        yb1[i] = np.array([0, 1, 0]) @ twc @ np.array([0, 0, 1])
        xb2[i] = np.array([1, 0, 0]) @ twd @ np.array([0, 0, 1])
        yb2[i] = np.array([0, 1, 0]) @ twd @ np.array([0, 0, 1])
        xb3[i] = np.array([1, 0, 0]) @ twe @ np.array([0, 0, 1])
        yb3[i] = np.array([0, 1, 0]) @ twe @ np.array([0, 0, 1])
        xb4[i] = np.array([1, 0, 0]) @ twf @ np.array([0, 0, 1])
        yb4[i] = np.array([0, 1, 0]) @ twf @ np.array([0, 0, 1])

        twg = get_g_matrix(0, xxj[i], yyj[i])
        tgh = get_g_matrix(tj[i], 0, 0)
        thi = get_g_matrix(0, L2/2, 0)
        thj = get_g_matrix(0, 0, L2/2)
        thk = get_g_matrix(0, -L2/2, 0)
        thl = get_g_matrix(0, 0, -L2/2)

        twh = twg @ tgh
        twi = twh @ thi
        twj = twh @ thj
        twk = twh @ thk
        twl = twh @ thl

        xj1[i] = np.array([1, 0, 0]) @ twi @ np.array([0, 0, 1])
        yj1[i] = np.array([0, 1, 0]) @ twi @ np.array([0, 0, 1])
        xj2[i] = np.array([1, 0, 0]) @ twj @ np.array([0, 0, 1])
        yj2[i] = np.array([0, 1, 0]) @ twj @ np.array([0, 0, 1])
        xj3[i] = np.array([1, 0, 0]) @ twk @ np.array([0, 0, 1])
        yj3[i] = np.array([0, 1, 0]) @ twk @ np.array([0, 0, 1])
        xj4[i] = np.array([1, 0, 0]) @ twl @ np.array([0, 0, 1])
        yj4[i] = np.array([0, 1, 0]) @ twl @ np.array([0, 0, 1])


    ###############################################
    # Getting data from pendulum angle trajectories.

    # xx1=xx
    # yy1=yy
    # xx2=xx1+L2*np.sin(t1)
    # yy2=yy1-L2*np.cos(t1)
    # xx3=xx1+L2*np.sin(t2)
    # yy3=yy1-L2*np.cos(t2)

    # # Leg 1
    # xx4=xx1-W/2*np.cos(t1)
    # yy4=yy1-W/2*np.sin(t1)
    # xx5=xx1+W/2*np.cos(t1)
    # yy5=yy1+W/2*np.sin(t1)
    # xx6=xx2+W/2*np.cos(t1)
    # yy6=yy2+W/2*np.sin(t1)
    # xx7=xx2-W/2*np.cos(t1)
    # yy7=yy2-W/2*np.sin(t1)

    # # Leg 2
    # xx8 =xx1-W/2*np.cos(t2)
    # yy8 =yy1-W/2*np.sin(t2)
    # xx9 =xx1+W/2*np.cos(t2)
    # yy9 =yy1+W/2*np.sin(t2)
    # xx10=xx3+W/2*np.cos(t2)
    # yy10=yy3+W/2*np.sin(t2)
    # xx11=xx3-W/2*np.cos(t2)
    # yy11=yy3-W/2*np.sin(t2)
    # N = len(theta_array[0]) # Need this for specifying length of simulation

    ####################################
    # Using these to specify axis limits.
    # xm=np.min(xx3)-0.5
    # xM=np.max(xx2)+0.5
    # ym=np.min(yy1)-0.5
    # yM=np.max(yy1)+0.5
    xm = -3
    xM =  3
    ym = -4
    yM =  4

    ###########################
    # Defining data dictionary.
    # Trajectories are here.
    data=[
        dict(x=xxb, y=yyb,
               mode='lines', name='Box',
               line=dict(width=2, color='red')
              ),
        dict(x=xxj, y=yyj,
               mode='lines', name='Jack 1',
               line=dict(width=2, color='blue')
              ),
        dict(x=xxj, y=yyj,
               mode='lines', name='Jack 2',
               line=dict(width=2, color='blue')
              ),
          dict(x=xj1, y=yj1,
               mode='markers', name='mass 1',
               line=dict(width=2, color='red')
              ),
          dict(x=xj2, y=yj2,
               mode='markers', name='mass 2',
               line=dict(width=2, color='green')
              ),
          dict(x=xj3, y=yj3,
               mode='markers', name='mass 3',
               line=dict(width=2, color='yellow')
              ),
          dict(x=xj4, y=yj4,
               mode='markers', name='mass 4',
               line=dict(width=2, color='blue')
              ),
          dict(x=xb1, y=yb1,
               mode='markers', name='box 1',
               line=dict(width=2, color='red')
              ),
          dict(x=xb2, y=yb2,
               mode='markers', name='box 2',
               line=dict(width=2, color='green')
              ),
          dict(x=xb3, y=yb3,
               mode='markers', name='box 3',
               line=dict(width=2, color='yellow')
              ),
          dict(x=xb4, y=yb4,
               mode='markers', name='box 4',
               line=dict(width=2, color='blue')
              ),
          # dict(x=xx2, y=yy2,
          #      mode='markers', name='Mass 1',
          #      line=dict(width=2, color='blue')
          #     ),
        #   dict(x=xx2, y=yy2,
        #        mode='lines', name='Leg 2',
        #        line=dict(width=2, color='blue')
        #       ),
          # dict(x=xx3, y=yy3,display(Markdown(r'$\lambda \nabla \phi_15=$'))
          #      mode='markers', name='Mass 2',
          #      line=dict(width=2, color='blue')
          #     ),
        #   dict(x=xx2, y=yy2,
        #        mode='markers', name='Pendulum 1 Traj',
        #        marker=dict(color="purple", size=2)
        #       ),
        #   dict(x=xx3, y=yy3,
        #        mode='markers', name='Pendulum 2 Traj',
        #        marker=dict(color="green", size=2)
        #       ),
        ]

    ################################
    # Preparing simulation layout.
    # Title and axis ranges are here.
    layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False,dtick=1),
                yaxis=dict(range=[ym, yM], autorange=False, zeroline=False,scaleanchor = "x",dtick=1),
                title='Jack and Box Simulation',
                hovermode='closest',
                updatemenus= [{'type': 'buttons',
                               'buttons': [{'label': 'Play','method': 'animate',
                                            'args': [None, {'frame': {'duration': T, 'redraw': False}}]},
                                           {'args': [[None], {'frame': {'duration': T, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],'label': 'Pause','method': 'animate'}
                                          ]
                              }]
               )

    ########################################
    # Defining the frames of the simulation.
    # This is what draws the lines from
    # joint to joint of the pendulum.
    frames=[dict(data=[dict(x=[xb1[k],xb2[k],xb3[k],xb4[k],xb1[k]],
                            y=[yb1[k],yb2[k],yb3[k],yb4[k],yb1[k]],
                            mode='lines',
                            line=dict(color='red', width=3)
                            ),
                       dict(x=[xj1[k],xj3[k]],
                            y=[yj1[k],yj3[k]],
                            mode='lines',
                            line=dict(color='blue', width=3)
                            ),
                       dict(x=[xj2[k],xj4[k]],
                            y=[yj2[k],yj4[k]],
                            mode='lines',
                            line=dict(color='blue', width=3)
                            ),
                       go.Scatter(
                            x=[xj1[k]],
                            y=[yj1[k]],
                            mode="markers",
                            marker=dict(color="red", size=12)),
                       go.Scatter(
                            x=[xj2[k]],
                            y=[yj2[k]],
                            mode="markers",
                            marker=dict(color="green", size=12)),
                       go.Scatter(
                            x=[xj3[k]],
                            y=[yj3[k]],
                            mode="markers",
                            marker=dict(color="yellow", size=12)),
                       go.Scatter(
                            x=[xj4[k]],
                            y=[yj4[k]],
                            mode="markers",
                            marker=dict(color="blue", size=12)),
                       go.Scatter(
                            x=[xb1[k]],
                            y=[yb1[k]],
                            mode="markers",
                            marker=dict(color="red", size=12)),
                       go.Scatter(
                            x=[xb2[k]],
                            y=[yb2[k]],
                            mode="markers",
                            marker=dict(color="green", size=12)),
                       go.Scatter(
                            x=[xb3[k]],
                            y=[yb3[k]],
                            mode="markers",
                            marker=dict(color="yellow", size=12)),
                       go.Scatter(
                            x=[xb4[k]],
                            y=[yb4[k]],
                            mode="markers",
                            marker=dict(color="blue", size=12)),
                      ]) for k in range(N)]

    #######################################
    # Putting it all together and plotting.
    figure1=dict(data=data, layout=layout, frames=frames)
    iplot(figure1)

In [ ]:

animate_system(traj_list, L1=4,L2=1,W=0.2,T=10)